## Importation du Dataset

Nous utilisons le jeu de données arXiv, contenant les métadonnées de plus de 2,7 millions d’articles scientifiques au format JSON.  
Chaque entrée contient des informations telles que : l’identifiant, le titre, les auteurs, le résumé, les catégories, et les références du papier.


In [ ]:
import json

path = '/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json'

data = []
with open(path, 'r') as f:
    for line in f:
        data.append(json.loads(line))  # Each line is a separate JSON object

print("Number of entries:", len(data))
print("First paper title:", data[0]['title'])

### Echantillon

In [ ]:
data[14434]

## Fusion du Titre avec l’Abstrait

Pour enrichir la représentation sémantique de chaque papier, nous avons fusionné le **titre** et l’**abstrait** en une seule chaîne de texte.  
Cela permet de mieux capturer le contenu global du document, car ces deux éléments contiennent l’essentiel de l’information descriptive sur le papier.

In [ ]:
import pandas as pd

def clean(text):
    return ' '.join(text.strip().split())


papers = pd.DataFrame([{
    'id': paper['id'],
    'text': clean(paper['title']) + ' ' + clean(paper['abstract'])
} for paper in data if paper.get('title') and paper.get('abstract')])

In [ ]:
papers.head()

## Prétraitement des Données

Avant d'utiliser les textes dans notre application, nous avons effectué plusieurs étapes de prétraitement pour nettoyer et standardiser les données. Ces étapes sont les suivantes :

1. **Mise en minuscule** : pour uniformiser le texte.  
2. **Suppression des expressions LaTeX** : élimination des éléments comme `\textbf{}`, `\n`, etc., fréquents dans les papiers scientifiques.  
3. **Normalisation des espaces** : suppression des espaces superflus.  
4. **Suppression des caractères non alphanumériques** (sauf ponctuation commune).  

À la fin de cette étape, chaque article est représenté par un **texte nettoyé**, accompagné de son **identifiant (ID)**.


In [ ]:
import re

def preprocess_data(text):

    #lowercase
    text = text.lower()

    #remove LaTeX formatting
    text = re.sub(r'\\[a-zA-Z]+', '', text)

    #Normalise whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    #Remove non-alphanumeric characters execpt for common ponctuation
    text = re.sub(r'[^a-z0-9.,;:!?\'"()\- ]', '', text)

    return text

In [ ]:
papers['text'] = papers['text'].apply(preprocess_data)

In [ ]:
print(papers.head())

### Sauvegarde du resultat

In [ ]:
papers.to_parquet("papers.parquet", index=False)

## Téléchargement du Modèle Sentence Transformers

Pour encoder les textes de manière sémantique, nous utilisons le modèle `all-MiniLM-L6-v2` de la bibliothèque **SentenceTransformers**.  
Ce modèle est léger, rapide, et offre de bonnes performances pour les tâches de similarité de texte.

### Installation de la bibliothèque

```bash
pip install sentence-transformers

In [ ]:
!pip install sentence_transformers hf_xet > /dev/null

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

2025-05-04 15:39:54.838939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746373194.864996     194 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746373194.872109     194 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Téléchargement des Résultats Prétraités

Le prétraitement des textes a été effectué en amont sur un TPU pour bénéficier d’une plus grande capacité de traitement.  
De même, la génération des embeddings a été réalisée sur GPU pour accélérer le calcul.

In [ ]:
!pip install gdown

In [4]:
file_id = '1-BuspKbP0168ys51JBhrrQfmj2nKf2A2'
file_name = 'papers.parquet'

# Download from Drive
!gdown --id {file_id} -O {file_name}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-BuspKbP0168ys51JBhrrQfmj2nKf2A2
From (redirected): https://drive.google.com/uc?id=1-BuspKbP0168ys51JBhrrQfmj2nKf2A2&confirm=t&uuid=8ac5dfdf-3969-4dd2-b073-f4bdb0f9a568
To: /kaggle/working/papers.parquet
100%|███████████████████████████████████████| 1.55G/1.55G [00:06<00:00, 243MB/s]


In [2]:
import pandas as pd

papers = pd.read_parquet('papers.parquet')

papers.head()

,id,text
0,0704.0001,calculation of prompt diphoton production cros...
1,0704.0002,sparsity-certifying graph decompositions we de...
2,0704.0003,the evolution of the earth-moon system based o...
3,0704.0004,a determinant of stirling cycle numbers counts...
4,0704.0005,from dyadic to in this paper we show how to ...


## Génération des Embeddings avec le Modèle

Une fois le modèle `all-MiniLM-L6-v2` chargé, nous pouvons générer les embeddings des textes nettoyés.  
Chaque texte (titre + résumé) sera transformé en un vecteur dense représentant sa signification sémantique.


In [3]:
embeddings = model.encode(
    papers['text'].tolist(),
    batch_size=128,
    show_progress_bar=True
)

Batches:   0%|          | 0/21255 [00:00<?, ?it/s]

In [9]:
embeddings.shape

(2720631, 384)

## Installation de FAISS

FAISS (**Facebook AI Similarity Search**) est une bibliothèque conçue pour effectuer des recherches rapides de similarité entre vecteurs à grande échelle.  
Elle est particulièrement adaptée à notre cas, où l’on compare des millions d’embeddings.


In [12]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.2 MB/s eta 0:00:00:00:0100:01


## Génération de l'Index FAISS avec IndexFlatL2

Pour effectuer la recherche de similarité, nous utilisons **FAISS** avec la méthode `IndexFlatL2`,  
qui repose sur la distance euclidienne (L2) pour comparer les vecteurs.

Cette méthode est simple et exacte, bien qu’un peu plus lente que les techniques approximatives,  
mais elle reste suffisante pour des projets de taille moyenne.

In [15]:
import faiss

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

### Sauvegarde de l'Index FAISS


In [18]:
faiss.write_index(index, "faiss_index.idx")

### Sauvegarde du Mapping des Identifiants


In [3]:
papers[['id']].to_parquet("id_mapping.parquet")

## Test de Recherche avec l'Index FAISS

Maintenant que nous avons sauvegardé notre index et notre mapping des identifiants,  
nous pouvons tester la recherche en utilisant une requête pour obtenir les articles les plus similaires.


In [35]:
prompt = "How to extract triplets from text"

query_emb = model.encode([prompt]).astype('float32')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
distances, indices = index.search(query_emb, 5)

In [34]:
top_papers = papers.iloc[indices[0]]

# Display the top matching papers
for i, row in top_papers.iterrows():
    print(f"Paper ID: {row['id']}")
    print(f"Title: {row['text']}")
    print("-" * 50)

Paper ID: 2205.05270
Title: relational triple extraction: one step is enough extracting relational triples from unstructured text is an essential task in natural language processing and knowledge graph construction. existing approaches usually contain two fundamental steps: (1) finding the boundary positions of head and tail entities; (2) concatenating specific tokens to form triples. however, nearly all previous methods suffer from the problem of error accumulation, i.e., the boundary recognition error of each entity in step (1) will be accumulated into the final combined triples. to solve the problem, in this paper, we introduce a fresh perspective to revisit the triple extraction task, and propose a simple but effective model, named directrel. specifically, the proposed model first generates candidate entities through enumerating token sequences in a sentence, and then transforms the triple extraction task into a linking problem on a "head  tail" bipartite graph. by doing so, all tr